In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
# Importamos el módulo con nuestras funciones
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.pca import todoJunto
# from src.pca import pca
# from src.pca.PCA_from_SVD import PCA_from_SVD
# from src.pca.PCA_from_SVD_jacobi_aprox import PCA_from_SVD_jacobi
from src.test_algorithms.err_relativo import err_relativo

#### Objetivo:

El objetivo de este notebook es comparar los resultados de:
+ PCA de scikit-learn 
+ la implementación de PCA usando la función SVD de numpy.
+ la implementación de PCA usando la implementación del algoritmo Jacobi one-sided para obtener la SVD

In [3]:
df = pd.read_csv('../../data/datos_limpios.csv', encoding = "L1")

- Tomamos de ejemplo, para estas pruebas, la sección 3 de la encuesta.

In [4]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']] ###Se seleccionan las preguntas que conforman esta sección

In [5]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()
seccion_3 = scaler.fit_transform(seccion_3)

## PCA de scikit-learn
Se obtienen los atributos que conforman la función componentes_principales del archivo pca.py

In [6]:
pca, var_exp, comp_prin, val_sing, pca_coef, eigenvalues = todoJunto.PCA_from_sklearn(seccion_3)

In [7]:
pca

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [8]:
comp_prin

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433591],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830593],
       [-0.54628676, -0.67975472]])

In [9]:
val_sing

array([1180.28600498,  782.39469765])

In [10]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [11]:
eigenvalues

array([2.48486074, 1.09189113])

In [12]:
var_exp

array([0.49697126, 0.21837784])

In [13]:
componentes = pd.DataFrame(pca_coef.T, index=['p10', 'p11', 
                                                         'p12', 'p13',
                                                         'p14', 
                                                         ])

In [14]:
####Se eligieron dos componentes principales, por lo que a continuación se muestran los dos componentes principales por cada pregunta
componentes = componentes.rename(columns={'index': 'preguntas', 0:'principal_component_1', 1:'principal_component_2'})
componentes

,principal_component_1,principal_component_2
p10,0.118769,-0.745795
p11,0.433563,0.450160
p12,0.487973,0.328743
p13,0.530490,-0.199727
p14,0.527617,-0.305260


## PCA usando SVD de numpy
Ahora se obtienen los PCA a través de la función `svd` del módulo `linalg` de `numpy` .

In [15]:
np_val_sing, np_pca_coef, np_comp_prin, np_var_exp = todoJunto.PCA_from_SVD(seccion_3)

In [16]:
np_val_sing # Valores singulares

array([1180.28600498,  782.39469765])

In [17]:
np_pca_coef # coeficientes

array([[ 0.11876861, -0.74579472, -0.64932849,  0.02964681, -0.0847221 ],
       [ 0.43356271,  0.45015978, -0.42239702,  0.64736557,  0.10898683]])

In [18]:
np_comp_prin # componentes principales)

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433591],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830593],
       [-0.54628676, -0.67975472]])

In [19]:
np_var_exp #Varianza explicada

array([0.49697126, 0.21837784])

Comparemos los resultados obtenidos 

In [20]:
#valores singulares
np.allclose(np_val_sing,val_sing)

True

Tenemos diferencias en los coeficientes:

In [21]:
# coeficientes
np.allclose(np_pca_coef,pca_coef)

False

In [22]:
# componentes
np.allclose(np_comp_prin,comp_prin)

True

In [23]:
# Varianza explicada
np.allclose(np_var_exp,var_exp)

True

## PCA usando SVD con aproximación por Jacobi one-sided
Ahora se obtienen los PCA a través de la aproximación a la  `svd` mediante el algoritmo Jacobi one-sided.

In [24]:
jac_val_sing, jac_pca_coef, jac_comp_prin, jac_var_exp = todoJunto.PCA_from_SVD_jacobi(seccion_3)

Comparemos los resultados obtenidos 

In [25]:
#valores singulares
np.allclose(jac_val_sing,val_sing)

False

In [26]:
jac_val_sing

array([1158.20422871,  742.90658598])

In [27]:
val_sing

array([1180.28600498,  782.39469765])

In [28]:
# coeficientes
np.allclose(jac_pca_coef,pca_coef)

False

In [29]:
jac_pca_coef

array([[ 0.2298819 ,  0.39996804,  0.30417493,  0.55149827,  0.62490572],
       [ 0.42540564, -0.24768259, -0.81573948,  0.15895795,  0.2588143 ]])

In [30]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [31]:
# componentes
np.allclose(jac_comp_prin,comp_prin)

False

In [32]:
# Varianza explicada
np.allclose(jac_var_exp,var_exp)

False

Vemos el error relativo en los valores singulares:

In [33]:
err = err_relativo(jac_val_sing, val_sing) 
err

array([0.01870884, 0.05047083])

In [34]:
np.linalg.cond(seccion_3)

3.5192351424081814